<b>Heartbeat event loop</b>

In [5]:
from IPython.display import Image

In [2]:
from pydantic import BaseModel, Field, ValidationError
import json

If an agent decides to update its memory, you do not want it to stop and wait for the user to say something before it takes its next action. You want it to think: "Okay, I updated my memory. Now I need to search the database. Okay, I found the data. Now I will formulate an answer for the user." <br>
To achieve this, MemGPT introduced the concept of the "Heartbeat" (which acts much like a clock cycle in a CPU). It fundamentally changes the agent from a reactive script (User speaks $\rightarrow$ AI speaks) into a continuous, autonomous background process.

1. <b>Re-engineering the action space</b> <br>
The first conceptual leap is that in this architecture, the LLM never just outputs raw text to the user. Everything is a tool call. Even sending a message to the user is executed via a send_message tool. <br>
To control the loop, every single tool in your registry must inherit a special parameter: request_heartbeat.

In [3]:
# 1. The Base Class for ALL tools
class AgenticTool(BaseModel):
    request_heartbeat: bool = Field(
        default=False, 
        description="Set to True to immediately execute another action. Set to False to wait for user input."
    )

# 2. A Tier 1 Memory Tool
class CoreMemoryReplace(AgenticTool):
    block_name: str = Field(description="The memory block to edit ('human' or 'persona').")
    old_text: str = Field(description="The text to replace.")
    new_text: str = Field(description="The new text to insert.")

# 3. The User Communication Tool
class SendMessage(AgenticTool):
    message: str = Field(description="The text to display to the human user.")

2. <b>The Heartbeat Event Loop</b> <br>
Rewrite the core orchestration loop. Instead of while step_count < max_steps, we implement an Inner Loop (the heartbeat) inside the Outer Loop (the user conversation).

In [4]:
def run_os_agent(user_input: str):
    # 1. The Outer Loop (Triggered by a user event)
    state_memory.append({"role": "user", "content": user_input})
    
    yield_to_user = False
    step_count = 0
    
    # 2. The Inner Loop (The Heartbeat)
    while not yield_to_user and step_count < 10:
        step_count += 1
        
        # The LLM reads the state and selects a tool
        llm_response = call_llm(state_memory)
        tool_name = llm_response["tool_name"]
        tool_args = llm_response["tool_args"] # E.g., {"request_heartbeat": True, "block_name": "human"...}
        
        # Execute the tool
        result = execute_tool(tool_name, tool_args)
        
        # Append the observation to the context window
        state_memory.append({
            "role": "system", 
            "content": f"Tool '{tool_name}' executed. Result: {result}"
        })
        
        # 3. The Heartbeat Evaluation
        if tool_args.get("request_heartbeat") is True:
            # The agent wants to keep working. We inject a "pulse".
            # This forces the LLM to immediately inference again without waiting for the human.
            print(f"[SYSTEM] Pulse {step_count}: Agent is chaining tasks...")
            state_memory.append({
                "role": "system", 
                "content": "Heartbeat. Please continue your operations."
            })
            
        else:
            # The agent set heartbeat to False (usually when calling send_message).
            # It has yielded the CPU thread back to the user.
            print(f"[SYSTEM] Task complete. Yielding to user.")
            yield_to_user = True
            
    return "Agent paused."

3. <i>Fail-safes and automatic heartbeats </i> <br>
The beauty of the heartbeat architecture is how elegantly it handles errors.If the LLM calls CoreMemoryReplace but misspells the block_name, the Python code catches the error. Instead of crashing, the system forces an automatic heartbeat. It appends the Python traceback to the state_memory and sets request_heartbeat = True under the hood. The LLM immediately wakes back up, reads its own error, and tries again.This creates a self-healing loop: $\text{Execute} \rightarrow \text{Fail} \rightarrow \text{Heartbeat} \rightarrow \text{Read Error} \rightarrow \text{Correct} \rightarrow \text{Execute}$.